# MIDI-to-audio Generation Notebook

<div style="text-align:center;">
<img src="../images/method.png" alt="Example Image" width="800" />
</div>


This notebook implements the inference for MIDI-to-audio generation. We demonstrate using the demo samples from the [webpage](https://nilsdem.github.io/control-transfer-diffusion/), but you can load your own midi files and timbre targets. 
Please note that although any MIDI file can be used, the model require samples from the datasets (or quite similar) for the timbre target.

Make sure to [download]() the pretrained models and place them in `./pretrained`. The only checkpoint available so far was trained on the [SLAKH 2100](http://www.slakh.com/) dataset.

### Some imports

In [ ]:
import os

os.chdir("..")  # eventually change working directory to root of the project

In [ ]:
import gin

gin.enter_interactive_mode()

from IPython.display import display, Audio
import pretty_midi
import matplotlib.pyplot as plt

import torch
import numpy as np
import librosa

torch.set_grad_enabled(False)

### Checkpoint setup

In [ ]:
# Import paths
folder = "./pretrained/slakh_midi/"
checkpoint_path = folder + "checkpoint.pt"
autoencoder_path = "./pretrained/AE_slakh.pt"
config = folder + "config.gin"

# GPU
device = "cuda:0"  # or "cpu"

### Load the model

In [ ]:
from diffusion.model import EDM_ADV

# Parse config
gin.parse_config_file(config)
SR = gin.query_parameter("%SR")
audio_length = gin.query_parameter("%X_LENGTH")

# Instantiate model
blender = EDM_ADV()

# Load checkpoints
state_dict = torch.load(checkpoint_path)["model_state"]
blender.load_state_dict(state_dict, strict=False)

emb_model = torch.jit.load(autoencoder_path).eval().to(device)

# Send to device
blender = blender.eval().to(device)

### Loading some audio files

In [ ]:
def load_audio(path, sr):
    audio_full, sr = librosa.load(path, sr=sr)
    audio = audio_full[:audio_length]
    audio = torch.from_numpy(audio).reshape(1, 1, -1) / audio.max()
    return audio


def process_audio(audio):
    audio = audio.to(device)
    z = emb_model.encode(audio)
    return z


def normalize(S):
    S = (S - np.min(S)) / (np.max(S) - np.min(S) + 1e-4)
    return S


def get_midi(file, audio_length, comp_ratio, sr):
    midi_data = pretty_midi.PrettyMIDI(file)
    length = audio_length
    tstart = 0
    tend = length
    midi_data.adjust_times([0, tend], [0, tend])

    pr = midi_data.get_piano_roll(
        times=np.linspace(0, audio_length / sr, audio_length // comp_ratio))

    pr = normalize(pr)
    return pr

In [ ]:
audio_file = './audios/midi/true/voice.wav'
midi_file = "./audios/midi/midi/voice.mid"

In [ ]:
audio1 = load_audio(audio_file, sr=SR)

print("Sample 1")
display(Audio(audio1.squeeze(), rate=SR))

# Compute embeddings and CQT
z1 = process_audio(audio1)

# Get audio length and compression ratio
audio_length = audio1.shape[-1]
comp_ratio = audio_length // z1.shape[-1]

# Load the midi

pr = get_midi(midi_file, audio_length, comp_ratio, SR)
plt.imshow(pr, aspect="auto", origin="lower")

### Generation

In [ ]:
nb_steps = 60  #Number of diffusion steps
guidance = 2.0  #Classifier free guidance strength

In [ ]:
# Compute structure representation
time_cond = torch.from_numpy(pr).reshape(1, pr.shape[-2],
                                         pr.shape[-1]).float().to(device)

time_cond = blender.encoder_time(time_cond)

# Compute timbre representation
zsem = blender.encoder(z1)

# Sample initial noise
x0 = torch.randn_like(z1)

xS = blender.sample(x0,
                    time_cond=time_cond,
                    zsem=zsem,
                    nb_step=nb_steps,
                    guidance=guidance,
                    guidance_type="time_cond")

audio_out = emb_model.decode(xS).cpu().numpy().squeeze()
display(Audio(audio_out, rate=SR))